In [1]:
from premise import *
from datapackage import Package
import brightway2 as bw
bw.projects.set_current("new")

In [2]:
clear_cache()

Cache folder cleared!


In [3]:
combinations = [
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 5, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00}
]

In [4]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2030},
    {"model": "remind", "pathway":"SSP5-Base", "year": 2080},
]

for model in ["cutoff", "consequential"]:

    for ei in [
        #"3.6",
        #"3.7",
        #"3.7.1",
        #"3.8",
        #"3.9",
        "3.9.1"

    ]:
        
        if ei not in ["3.9", "3.9.1"]:
            bw.projects.set_current("new4")
        else:
            bw.projects.set_current("ei39")

        for source in [
            "brightway",
            #"ecospold"
        ]:
            for export in [
                "brightway",
                "superstructure",
                "simapro",
                "matrices",
                "datapackage"
            ]:

                ndb = NewDatabase(
                        scenarios = scenarios,
                        source_db= f"ecoinvent {ei} {model}" if source == "brightway" else None,
                        source_type=source,
                        source_file_path=f"/Users/romain/Documents/ecoinvent {ei}_cutoff_ecoSpold02/datasets" if source == "ecospold" else None, # <-- this is NEW
                        source_version=ei,
                        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
                        system_model=model,
                        system_args=combinations[0],
                        additional_inventories= [ # <-- this is NEW
                            {"filepath": "/Users/romain/GitHub/premise/premise/data/additional_inventories/lci-PV.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
                    ] # <-- this is NEW
                )
                ndb.generate_scenario_report()
                ndb.update_all()

                print("#############################################################################################################")
                print(ei, source, export)

                #ndb.update_buses()
                #ndb.update_cars()
                #ndb.update_trucks()
                #ndb.update_two_wheelers()


                if export == "brightway":
                    ndb.write_db_to_brightway([
                        "test1",
                        "test2"
                        ]
                    )
                    lca = bw.LCA({bw.Database("test1").random(): 1})
                    lca.lci()
                    lca = bw.LCA({bw.Database("test2").random(): 1})
                    lca.lci()
                elif export == "superstructure":
                    ndb.write_superstructure_db_to_brightway("test_super")
                elif export == "simapro":
                    ndb.write_db_to_simapro()
                elif export == "matrices":
                    ndb.write_db_to_matrices()
                else:
                    ndb.write_datapackage()
                
            
            


premise v.(1, 5, 0, 'alpha')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+----

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21238/21238 [00:00<00:00, 291310.36it/s]


Adding exchange data to activities


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 674593/674593 [00:58<00:00, 11469.66it/s]


Filling out exchange data


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18856/18856 [00:03<00:00, 5752.10it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Correct missing flow categories for biosphere exchanges
Remove empty exchanges.
Remove uncertainty data.
Done!

////////////////// IMPORTING DEFAULT INVENTORIES ///////////////////
Cannot find cached inventories. Will create them now for next time...
Importing default inventories...

Extracted 1 worksheets in 0.17 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Extracted 1 worksheets in 0.01 seconds
Migrating to 3.8 first
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: mig

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:32


Title: Writing activities to SQLite3 database:
  Started: 03/21/2023 14:15:19
  Finished: 03/21/2023 14:15:52
  Total time elapsed: 00:00:32
  CPU %: 93.90
  Memory %: 27.26


ValueError: Invalid amount in exchange {'uncertainty type': 0, 'amount': nan, 'product': 'diesel, low-sulfur', 'name': 'diesel production, low-sulfur, petroleum refinery operation', 'unit': 'kilogram', 'location': 'Europe without Switzerland', 'type': 'technosphere', 'input': ('test1', '551e7cfa8d88ff94706e4cca1dd74c8c'), 'output': ('test1', '55d3f1d40ea94e6ea9a8af9ce734d639')}

In [6]:
units = []
for ds in ndb.database:
    for e in ds["exchanges"]:
        if e["unit"] not in units:
            units.append(e["unit"])

In [7]:
units

['kilogram',
 'kilowatt hour',
 'megajoule',
 'unit',
 'ton kilometer',
 'cubic meter',
 'meter',
 'square meter',
 'square meter-year',
 'kilometer',
 'kg*day',
 'hour',
 'hectare',
 'kilo Becquerel',
 'cubic meter-year',
 'Sm3',
 'litre',
 'meter-year',
 'person kilometer',
 'kilometer-year',
 'guest night']

In [4]:
ndb.scenarios[1]["iam data"].other_vars.sel(variables="Temperature|Global Mean", region="World")

<xarray.DataArray 'value' (year: 19)>
array([0.882371, 0.894388, 0.993749, 1.114845, 1.239434, 1.35875 ,
       1.502813, 1.679966, 1.908864, 2.186055, 2.440907, 2.69551 ,
       3.200126, 3.682789, 4.123068, 4.513573, 4.860211, 5.423776,
       5.879446])
Coordinates:
    region     <U5 'World'
    variables  <U23 'Temperature|Global Mean'
  * year       (year) int64 2005 2010 2015 2020 2025 ... 2100 2110 2130 2150

In [5]:
for ds in bw.Database("ecoinvent 3.8 cutoff"):
    for e in ds.exchanges():
        if e["type"] == "biosphere" and any(i in e["name"] for i in ["HFC", "HCFC", "CFC", "Halon"]):
            print(ds["name"], e["name"])

uranium production, centrifuge, enriched 4.2% Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
transport, trolleybus Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
electricity production, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
electricity production, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
transport, freight, lorry with refrigeration machine, 3.5-7.5 ton, EURO3, R134a refrigerant, cooling Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
heat and power co-generation, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
transport, freight, lorry with refrigeration machine, 3.5-7.5 ton, EURO4, R134a refrigerant, cooling Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
electricity production, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
heat and power co-generation, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
electricity production, hard coal Ethane, 1,1,1-trichloro-, HCFC-140
uranium production, diffusion, enriched 4.0% Ethane, 1,2-dichloro-1,1,2,2-tetrafluoro-, CFC-114
electricity production, hard coal Ethane, 1,1,1-tric

In [1]:
from premise.geomap import Geomap
geo = Geomap("remind")

In [ ]:
geo.geo.add_definitions

In [5]:
for ds in ndb.database:
    if ds["name"] == "market for heat, from steam, in chemical industry":
        print(ds["name"], ds["reference product"], ds["location"], ds["unit"])

market for heat, from steam, in chemical industry heat, from steam, in chemical industry RER megajoule
market for heat, from steam, in chemical industry heat, from steam, in chemical industry RoW megajoule


In [6]:
for ds in ndb.scenarios[0]["database"]:
    for e in ds["exchanges"]:
        if e["input"][0] == '4b344fb0e0cd348af7cf9007812638b0':
            print(e)

KeyError: 'input'

In [4]:
from premise.activity_maps import InventorySet

i = InventorySet(
    ndb.database,
)

In [7]:
i.generate_fuel_map()

{'hard coal': {'market for hard coal', 'market for hard coal, run-of-mine'},
 'lignite': {'market for lignite'},
 'petroleum coke': {'market for petroleum coke'},
 'wood pellet': {'market for wood pellet, measured as dry mass'},
 'wood chips': {'market for wood chips and particles, willow',
  'market for wood chips, dry, measured as dry mass',
  'market for wood chips, from post-consumer wood, measured as dry mass',
  'market for wood chips, wet, measured as dry mass'},
 'natural gas': {'natural gas production',
  'natural gas production, liquefied',
  'natural gas production, unprocessed, at extraction',
  'natural gas, at production',
  'natural gas, at production offshore',
  'natural gas, at production onshore'},
 'light fuel oil': {'light fuel oil production, from waste polyethylene',
  'light fuel oil production, petroleum refinery operation',
  'light fuel oil, import from Europe',
  'light fuel oil, import from RoW'},
 'biogas': {'Biogas, burned in power plant/NGCC, no CCS',
  